In [1]:
#!/usr/bin/env python3
from collections import deque

def solve():
    # Read input lines (each is something like "805A", "983A", etc.)
    with open("input.txt","r") as f:
        lines = [line.strip() for line in f if line.strip()]

    # We'll parse these lines into a list of codes.
    codes = lines

    # 1) Define the keypad layouts as small grids (button->(r,c)), and (r,c)->button where needed.
    #    We have 3 keypad types: top-level (the same layout for you and each robot),
    #    plus the numeric keypad.

    # 1a) Numeric keypad:
    #   r=0:   7  8  9
    #   r=1:   4  5  6
    #   r=2:   1  2  3
    #   r=3:      0  A
    numeric_positions = {
        '7': (0, 0),
        '8': (0, 1),
        '9': (0, 2),
        '4': (1, 0),
        '5': (1, 1),
        '6': (1, 2),
        '1': (2, 0),
        '2': (2, 1),
        '3': (2, 2),
        '0': (3, 1),
        'A': (3, 2),
    }
    numeric_positions_rev = {v: k for k,v in numeric_positions.items()}

    # 1b) Directional keypad for robots / you:
    #   r=0:   (gap) '^'  'A'
    #   r=1:   '<'   'v'   '>'
    dir_positions = {
        '^': (0, 1),
        'A': (0, 2),
        '<': (1, 0),
        'v': (1, 1),
        '>': (1, 2),
    }
    dir_positions_rev = {v:k for k,v in dir_positions.items()}

    # A helper to see if a move is valid on a given keypad:
    # We define direction deltas for '^','v','<','>':
    deltas = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1),
    }

    # For the numeric keypad, pressing '^' means "move up" from posNumeric, etc.
    # For a directional keypad, pressing '^' means "move up" from that position.
    # We never want to aim into a gap or off-grid, so if the new position isn't recognized, return None.

    def move_on_dir_keypad(pos_button, direction):
        """Move from pos_button in direction (one of '^','v','<','>') on a directional keypad.
           Return new_button or None if invalid."""
        (r, c) = dir_positions[pos_button]
        (dr, dc) = deltas[direction]
        r2, c2 = r+dr, c+dc
        if (r2, c2) in dir_positions_rev:
            return dir_positions_rev[(r2,c2)]
        else:
            return None  # invalid / gap

    def move_on_numeric_keypad(pos_button, direction):
        """Same, but for the numeric keypad."""
        (r, c) = numeric_positions[pos_button]
        (dr, dc) = deltas[direction]
        r2, c2 = r+dr, c+dc
        if (r2, c2) in numeric_positions_rev:
            return numeric_positions_rev[(r2,c2)]
        else:
            return None  # invalid / gap

    # We'll do a BFS in 5D: (posYou, posRobot2, posRobot1, posNumeric, typed_index).
    # Each BFS "step" is exactly 1 press on your keypad.

    def get_min_press_count(code):
        """
        Return the fewest number of your (top-level) keypad presses needed 
        to type all characters in `code` (e.g. "805A") on the numeric keypad.
        """

        # Convert code into a list of chars, e.g. "805A" -> ['8','0','5','A']
        code_list = list(code)
        code_len = len(code_list)

        # Initial state:
        start_state = ('A', 'A', 'A', 'A', 0)  
        # meaning: your arm at 'A', Robot#2's arm at 'A', Robot#1's arm at 'A', numeric arm at 'A', typed_index=0.

        # BFS queue and visited set
        queue = deque()
        queue.append(start_state)
        visited = set([start_state])

        steps = 0  # how many top-level presses we've done so far in this BFS "layer"

        # We’ll do a standard BFS by layers. 
        # We'll track how many states are in the current layer, and increment steps after we finish each layer.
        while queue:
            layer_size = len(queue)
            for _ in range(layer_size):
                (posYou, posR2, posR1, posNum, tidx) = queue.popleft()

                # If we've typed the entire code, we are done.
                if tidx == code_len:
                    return steps  # the BFS level is the minimal top-level presses

                # We can press one of 5 possible buttons on your keypad: '^','v','<','>','A'
                possible_actions = ['^','v','<','>','A']
                for action in possible_actions:
                    # Attempt to apply that action from the current state
                    next_state = apply_top_level_action(
                        posYou, posR2, posR1, posNum, tidx, action, code_list
                    )
                    if next_state is None:
                        continue  # invalid action or leads to gap / mismatch
                    if next_state not in visited:
                        visited.add(next_state)
                        queue.append(next_state)

            steps += 1

        # If somehow we exhausted BFS without finishing code, it means it's impossible.
        # The puzzle states it's always possible, but we'll return a large number or raise an error.
        return 999999999

    def apply_top_level_action(posYou, posR2, posR1, posNum, tidx, action, code_list):
        """
        Applies a single top-level keypad press (one of '^','v','<','>','A')
        and returns the new state or None if invalid.
        
        - If action is an arrow, we move posYou on the top-level keypad (if possible).
        - If action == 'A', we "press" the button that posYou is pointing at, 
          which instructs Robot #2 to do something. 
        """
        if action in deltas:  # i.e. '^','v','<','>'
            # Move your arm on the top-level keypad
            newPosYou = move_on_dir_keypad(posYou, action)
            if newPosYou is None:
                return None  # invalid / gap
            # Everything else unchanged
            return (newPosYou, posR2, posR1, posNum, tidx)

        else:
            # action == 'A': We press the button your arm is currently aiming at
            buttonYou = posYou  # Could be '^','v','<','>','A'

            if buttonYou in deltas:
                # Means we instruct Robot #2 to move in that direction
                newPosR2 = move_on_dir_keypad(posR2, buttonYou)
                if newPosR2 is None:
                    return None  # gap or invalid
                return (posYou, newPosR2, posR1, posNum, tidx)

            else:
                # buttonYou == 'A' => We instruct Robot #2 to "press" 
                # (the button Robot #2 is currently aiming at).
                buttonR2 = posR2
                if buttonR2 in deltas:
                    # Robot #2 moves Robot #1 in that direction
                    newPosR1 = move_on_dir_keypad(posR1, buttonR2)
                    if newPosR1 is None:
                        return None
                    return (posYou, posR2, newPosR1, posNum, tidx)
                else:
                    # buttonR2 == 'A' => Robot #2 "presses", 
                    # which means Robot #1 either moves numeric arm or presses numeric button
                    buttonR1 = posR1
                    if buttonR1 in deltas:
                        # Move numeric arm
                        newPosNum = move_on_numeric_keypad(posNum, buttonR1)
                        if newPosNum is None:
                            return None
                        return (posYou, posR2, posR1, newPosNum, tidx)
                    else:
                        # buttonR1 == 'A' => Robot #1 "presses" the numeric button
                        # That means we attempt to type whatever posNum is (digit or 'A')
                        this_char = posNum  # e.g. '8','0','5','A', etc.
                        needed_char = code_list[tidx]
                        if this_char == needed_char:
                            # typed the correct next char
                            new_tidx = tidx + 1
                            return (posYou, posR2, posR1, posNum, new_tidx)
                        else:
                            return None  # typed a mismatch
        # Should not get here
        return None

    # 2) Now we just compute the sum of complexities for each line in input.
    #    Complexity = (#top-level-presses) * (integer part ignoring leading zeros)

    total_sum = 0
    for code in codes:
        # numeric part is everything except the trailing 'A' if present, ignoring leading zeros
        # (If the puzzle might have codes always ending with 'A', we can do so:
        if code.endswith('A'):
            digits_str = code[:-1]
        else:
            digits_str = code
        if digits_str == "":
            val = 0
        else:
            val = int(digits_str)  # automatically ignores leading zeros

        # BFS to find the minimal # of your presses to type this code
        length = get_min_press_count(code)

        complexity = length * val
        total_sum += complexity

    print(total_sum)

if __name__ == "__main__":
    solve()

492496
